In [1]:
import numpy as np
import matplotlib.pylab as plt

import glob
import re

In [2]:
def peak_Gauss(x, position, fwhm, amplitude,
               base_height, base_slope):
    amplitude = amplitude * 2/fwhm * np.sqrt(np.log(2)/np.pi)
    gauss = np.exp(-4*np.log(2)/fwhm**2 * (x-position)**2 )
    base_line = base_height + base_slope * x
    return amplitude*gauss + base_line

def peak_Lorentzien(x, position, fwhm, amplitude, base_height, base_slope):
    base_line = base_height + base_slope * x
    return 2*amplitude/np.pi/fwhm/(1 + 4*(np.sqrt(2)-1)/fwhm**2*(x-position)**2) + base_line

def peak_pseudoVoigt(x, position, fwhm, amplitude, base_height, base_slope, eta):
    return (1-eta)*peak_Gauss(x, position, fwhm, amplitude, base_height, base_slope) + \
           eta*peak_Lorentzien(x, position, fwhm, amplitude, 0, 0)


def fit_peak(x, y, function='pseudoVoigt'):
    ''' Fit a peak function on the given x, y data
        function 'Lorentz', 'pseudoVoigt' or 'Gauss'
        
        Returns:
            center position
            FWHM
            peak(x): function of the fitted peak 
    '''
    x, y = np.asarray(x), np.asarray(y)

    p0 = [x[np.argmax(y)], # center
          x[y>(y.min()+y.ptp()/2)].ptp(), # estimated width
          y.ptp(), # amplitude, peak to peak, i.e. max - min
          y.min(), # base height
          0]       # base slope

    if function == 'Lorentz':
        popt, pcov = curve_fit(peak_Lorentzien, x, y, p0=p0)
        #plt.plot(x, peak_Lorentzien(x, *popt), 'r');
        fitted_function = lambda u: peak_Lorentzien(u, *popt)
        FWHM = 2*popt[1]
        
    elif function == 'pseudoVoigt':
        p0.append(0.5) # eta
        popt, pcov = curve_fit(peak_pseudoVoigt, x, y, p0=p0)
        fitted_function = lambda u: peak_pseudoVoigt(u, *popt)
        FWHM = 2*popt[1] # no!!
        
    else: # Gauss
        popt, pcov = curve_fit(peak_Gauss, x, y, p0=p0)
        #plt.plot(x, peak_Gauss(x, *popt), 'r');
        fitted_function = lambda u: peak_Gauss(u, *popt)
        FWHM = 2*np.sqrt(2*np.log(2))*popt[1]
    
    
    return popt[0], np.abs(FWHM), fitted_function

In [3]:
''' CSV Tools
'''

def read_csv(path):
    skip_header = 34
    data = np.genfromtxt(path,
                         skip_header=skip_header,
                         delimiter=',')
    data_dico = dict()
    data_dico['deuxtheta'] = data[:, 0]
    data_dico['I'] = data[:, 1]
    return data_dico


def get_field(csv, fieldname):
    '''Extract information form the csv file (string) for the asked fieldname'''
    line = next(line for line in csv if line.startswith(fieldname))
    
    line = line.strip().split(',')[1:]
    
    if len(line) == 1:
        return float(line[0])
    else:
        return line


class Measure():

    def __init__(self, path):
        self.path = path
        
        m = read_csv(path)
        self.deuxtheta = m['deuxtheta']
        self.I = m['I']
        
        self.fit(plot=False)
        
        # Extract measurement info
        with open(path, 'r') as f:
            csv = f.readlines()
            self.phi = get_field(csv, 'Phi')
            self.psi = phi = get_field(csv, 'Psi')
            
        # Extract measurement name
        name = path.split('/')[-1]
        words_to_remove = ['moved', '.csv', '2theta_omega']
        for word in words_to_remove:
            name = name.replace(word, '').strip('._')
        self.name = name
        
        # Extract hkl values
        hkl_pattern = '(-?\d)(-?\d)(-?\d)'
        self.hkl = [int(i) for i in re.findall(hkl_pattern, self.name)[0]]
        
        self.d_hkl_theo = distance_from_hkl(self.hkl, 'Cu' if 'Cu' in self.name else 'Nb')
        
    def plot(self):
        plt.plot(self.deuxtheta, self.I, label=self.name);
        plt.xlabel('deux theta (deg)');
        plt.ylabel('I');
        plt.title(self.name)
        
        
    def fit(self, plot=True, function='pseudoVoigt'):
        
        try:
            x0, fwhm, fitfunction = fit_peak(self.deuxtheta, self.I,
                                      function=function)
        except RuntimeError:
            x0, fwhm, fitfunction = np.NaN, np.NaN, lambda x:0
            
        self.x0 = x0
        self.fwhm = fwhm
        self.fitfunction = fitfunction
        
        self.d_fit = distance_from_Bragg(x0)
        
        if plot:
            ax1 = plt.subplot(2, 1, 1);
            self.plot();
            plt.legend();

            plt.plot(self.deuxtheta, self.fitfunction(self.deuxtheta), 'r', alpha=0.5)
            plt.plot(self.x0, self.fitfunction(self.x0), 'or', alpha=0.5);
            
            deuxtheta_theo = (self.deuxtheta[0]+self.deuxtheta[-1])/2
            plt.axvline(x=deuxtheta_theo)
            
            # residue
            plt.subplot(2, 1, 2, sharex=ax1);
            plt.axhline(y=0, linewidth=1, color='black')
            plt.plot(self.deuxtheta, self.I-self.fitfunction(self.deuxtheta), 'r', alpha=0.5)
            

In [4]:
from scipy.optimize import curve_fit

def peak_Gauss(x, x0, sigma, A, y_min=0, slope=0):
    return A*np.exp(-(x-x0)**2/2/sigma**2) + y_min + slope * x


def fit_peak(x, y):
    ''' Fit a peak function on the given x, y data
        function 'Lorentz' or 'Gauss'
        
        Returns:
            center position
            FWHM
            peak(x): function of the fitted peak 
    '''
    p0 = [x[np.argmax(y)], # center
          x[y>(y.min()+y.ptp()/2)].ptp(), # estimated width
          y.ptp()], # amplitude, peak to peak, i.e. max - min
       #   y.min(), # base height
       #   0]   # base slope

    try:
        popt, pcov = curve_fit(peak_Gauss, x, y, p0=p0)
    except RuntimeError:
        popt = [np.NaN, 0]
    #plt.plot(x, peak_Gauss(x, *popt), 'r');
    #fitted_function = lambda u: peak_Gauss(u, *popt)
    #FWHM = 2*np.sqrt(2*np.log(2))*popt[1]

    return popt[0]